## What is a Sudoku?

[Sudoku](https://en.wikipedia.org/wiki/Sudoku) is one of the world's most popular puzzles. It consists of a 9x9 grid, and the objective is to fill the grid with digits in such a way that each row, each column, and each of the 9 principal 3x3 subsquares contains all of the digits from 1 to 9. The detailed rules can be found, for example, [here](https://www.conceptispuzzles.com/index.aspx?uri=puzzle/sudoku/rules).

The puzzle is given as partially completed grid, and the goal is to fill in the missing numbers. Below is an example of such grid. 

![alt-text](./images/sudoku-easy.png)

## Goal of this project

The main goal of this project is to build an itelligent agent that will solve every sudodu while introducing you to two powerful techniques that are used throughout the field of AI:

- **Constraint Propagation**

    When trying to solve a problem, you'll find that there are some local constraints to each square. These constraints help you narrow the possibilities for the answer, which can be very helpful. We will learn to extract the maximum information out of these constraints in order to get closer to our solution. Additionally, you'll see how we can repeatedly apply simple constraints to iteratively narrow the search space of possible sulutions. Constraint propagation can be used to solve a variety of problems such as calendar scheduling, and cryptographic puzzles.
- **Search**

    In the process of problem solving, we may get to the point where two or more possiblilities are available. What do we do? What if we branch out and consider both of them? Maybe one of them will lead us to a position in which three or more possibilities are available. Then we cana branch out again. At the end, we can create a whole tree of possibilities and find ways to traverse the tree until we find our solution. This is an example of how search can be used.

These ideas may seem simple and they're actually intended to be! Through this lesson you'll see how AI is really composed of very simple ideas can can but put together to solve complex problems. Throughout this lesson, we challenge you to think of how you can apply these ideas to build AI agents to solve other puzzles and problems in your worlds.


# Naming Conventions

## Rows and Columns

Since we're writing an agent to solve the Sudoku puzzle, let's start by laballing rows and columns.

- The rows will be labelled by the letters A, B, C, D, E, F, G, H, I
- The columns will be labelled by the numbers 1, 2, 3, 4, 5, 6, 7, 8, 9. Here we can see the unsolved and solve puzles with the labels for the rows and columns.
- The 3x3 squares won't be labelled, but in the diagram, they can be seen with alternating colors of grey and white.

![](./images/labels.png)

## Boxes, Units and Peers

And let's start naming the important elements created by these rows and columns that are relevant to solving a Sudoku:
- The individual squares at the intersection of rows and columns will be called `boxes`. These boxes will have labels 'A1', 'A2', ..., 'I9'.
- The complete rows, columns, and 3x3 squares will be called `units`. Thus, each unit is a set of 9 boxes, and there are 27 units in total.
- For a particular box (such as 'A1'), its `peers` will be all other boxes that belong to a common unit (namely, those that belong to the same row, colun, or 3x3 square).

Let's see an example. In the grids below, the set of highlighted boxes represent `units`. Each grid shows a different `peer` of the box at E3.

![](./images/peers.png)

# Encoding the Board

![](./images/labels.png)


Now, in order to implement an agent, let's start by coding the board in Python. Then, we'll code the necessary functions to solve the Sudoku. We'll record the puzzles in two ways — as a `string` and as a `dictionary`.

The string will consist of a concatenation of all the readings of the digits in the rows, taking the rows from top to bottom. If the puzzle is not solved, we can use a `.` as a placeholder for an empty box.

For example, the unsolved puzzle at the above left will be written as:
`..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..`

And the solved puzzle at the above right, will be recorded as:
`483921657967345821251876493548132976729564138136798245372689514814253769695417382`

We'll implement the dictionary as follows. The keys will be string corresponding to the boxes - namely, `A1`, `A2`, ..., `I9`. The values will either be the digit in each box (if there is one) or a `'.'` (if not).

So, let's get started. First we'll record rows and columns as strings.

In [2]:
rows = 'ABCDEFGHI'
cols = '123456789'

We'll start by writing a help function, `cross(a, b)`, which given two strings — `a` and `b` — will return the list formed by all the possible concatenations of a letter `s` in string `a` with a letter `t` in string `b`.

So `cross('abc', 'def')` will return `['ad', 'ae', 'af', 'bd', 'be', 'bf', 'cd', 'ce', 'cf']`

In [3]:
def cross(a, b):
    return [s+t for s in a for t in b]

Now, to create the labels of the boxes:

In [4]:
boxes = cross(rows, cols)
print(boxes)

['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9']


And for the units and peers:

In [20]:
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]
unitlist = row_units + column_units + square_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)


## Implement `grid_values()`

**A function to convert the string representation of a puzzle into a dictionary from.**


Recall that for the string:

`
'..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
`

...we'd like to return the dictionary:

```
{
    'A1': '.',
    'A2': '.',
    'A3': '3',
    ...
    'I9': '.'
}
```

In [25]:
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '.' value for empties.
    
    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form.
    """
    return dict(zip(boxes, grid))

## Implement `display()`
**A function to display sudoku board**

In [40]:
def display(values):
    """Display a sudoku board
    Args:
        values: Sudoku in dictionary form.
    """
    width = max(len(s) for _, s in values) + 1
    line = '+'.join(['-'*3*width]*3)
    for r in rows:
        print(''.join(values[r+c].center(width) + ('|' if c in '36' else '') for c in cols ) )
        if r in 'CF':
            print(line)

In [42]:
# Sanity check
values = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
display(values)

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 
